In [2]:
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
import pandas as pd
from tqdm.auto import tqdm
import pickle

In [3]:
df0 = pd.read_json('data/spiderbase_0.json')
df0.rename(columns={"link": "piwigo_link"}, inplace=True)
df1 = pd.read_json('data/spiderbase_1.json')
df1.rename(columns={"link": "nmbe_link"}, inplace=True)

In [4]:
# merge split family classes
df0.loc[:,'family'] = [x.split(' ')[0] for x in df0.family]

In [5]:
data0 = []
for species_name in tqdm(df0.species):
    fname = f'data/image_data_0/{species_name.replace(" ","-").lower()}.pkl'
    with open(fname,'rb') as f:
        tdict = pickle.load(f).copy()
        tdict['habitus'] = tdict.pop('general')
        tdict['pedipalp'] = tdict.pop('palpe')
        data0.append(tdict)
    
df0['piwigo'] = data0

In [6]:
data1 = []
for species_name in tqdm(df1.species):
    fname = f'data/image_data_1/{species_name.replace(" ","-").lower()}.pkl'
    with open(fname,'rb') as f:
        data1.append(pickle.load(f).copy())
    
df1['nmbe'] = data1

In [7]:
empty_dict = {'habitus': [], 'epigyne': [], 'vulve': [], 'pedipalp': [], 'other': []}

In [34]:
dd = df1.copy()
dd.loc[:, 'piwigo_link'] = np.full(len(dd), 'NONE')
dd.loc[:, 'piwigo'] = np.full(len(dd), empty_dict)

new_df = []
for family, genus, species, piwigo_link, piwigo in tqdm(zip(*df0.values.T), total = len(df0.values)):
    if species == genus:
        if '-x' in genus:
            genus_part = genus.split('-')[0].lower()
            subset = dd.loc[(dd.family == family) & ([genus_part in x for x in dd.genus])].copy()
            subset = subset.loc[~np.in1d(subset.genus, df0.loc[(df0.family == family) & ([genus_part in x for x in df0.genus])].genus)]
            subset = subset.loc[~np.in1d(subset.species, df0.loc[(df0.family == family) & ([genus_part in x for x in df0.genus])].species)]
            subset['piwigo_link'] = np.full(len(subset), piwigo_link)
            subset['piwigo'] = np.full(len(subset), piwigo)
            new_df.append(subset)
        elif 'x-' in genus:
            genus_part = genus.split('-')[1].lower()
            subset = dd.loc[(dd.family == family) & ([genus_part in x for x in dd.genus])].copy()
            subset = subset.loc[~np.in1d(subset.genus, df0.loc[(df0.family == family) & ([genus_part in x for x in df0.genus])].genus)]
            subset = subset.loc[~np.in1d(subset.species, df0.loc[(df0.family == family) & ([genus_part in x for x in df0.genus])].species)]
            subset['piwigo_link'] = np.full(len(subset), piwigo_link)
            subset['piwigo'] = np.full(len(subset), piwigo)
            new_df.append(subset)
        else:
            subset = dd.loc[(dd.family == family) & (dd.genus == genus)].copy()
            subset = subset.loc[~np.in1d(subset.species, df0.loc[(df0.family == family) & (df0.genus == genus)].species)]
            subset = subset.loc[['x-' not in x and '-x' not in x for x in subset.genus]]
            subset['piwigo_link'] = np.full(len(subset), piwigo_link)
            subset['piwigo'] = np.full(len(subset), piwigo)
            new_df.append(subset)
    else:
        subset = dd.loc[(dd.family == family) & (dd.genus == genus) & (dd.species == species)]
        if len(subset) == 1:
            mask = (dd.family == family) & (dd.genus == genus) & (dd.species == species)
            dd.loc[mask, 'piwigo_link'] = np.full(np.count_nonzero(mask), piwigo_link)
            dd.loc[mask, 'piwigo'] = np.full(np.count_nonzero(mask), piwigo)
        elif len(subset) == 0:
            mask = (df0.family == family) & (df0.genus == genus) & (df0.species == species)
            subset = df0.loc[mask].copy()
            subset['nmbe_link'] = np.full(len(subset), 'NONE')
            subset['nmbe'] = np.full(len(subset), empty_dict)
            new_df.append(subset)
        else:
            print(f'Error with: {family}, {genus}, {species}')

new_df = pd.concat(new_df, ignore_index=True)
database = pd.concat([dd, new_df], ignore_index=True).sort_values(by=['family', 'genus', 'species']).reset_index(drop=True)

# del dd
# del subset
# del new_df

In [39]:
database.to_pickle('data/full_database.pkl')

In [40]:
database = pd.read_pickle('data/full_database.pkl')